In [1]:
%matplotlib inline
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import time
from elasticsearch import Elasticsearch, helpers

In [2]:
last_days=120
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

def time_filter(indices,pat=r'jobs_archive_', last_days=1):
    if last_days == 0:
        return ["jobs_archive_*"]
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(pat, '', i).rstrip()
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if diff.days < last_days:
            filtered.append(i.rstrip())
    return filtered

def get_indices(index="jobs_archive_*",pat=r'jobs_archive_',last_days=last_days):
    #indices_ja = es.cat.indices(index="jobs_archive_*", h="index", request_timeout=600).split('\n')
    indices = es.cat.indices(index=index, h="index", request_timeout=600).split('\n')
    indices = sorted(indices)
    indices = [x for x in indices if x != '' and not x.endswith('_reindexed')]
    #print("indices is",indices)
    if index=="boinc":
        ind=indices
    else:
        ind = time_filter(indices,pat=pat,last_days=last_days)
        print ("number of indices:", len(ind))
    ind = ','.join(ind)
    print(ind)
    return ind
ind_ja=get_indices()
ind_boinc=get_indices(index="boinc",pat=r'boinc_')

number of indices: 120
jobs_archive_2017-09-14,jobs_archive_2017-09-15,jobs_archive_2017-09-16,jobs_archive_2017-09-17,jobs_archive_2017-09-18,jobs_archive_2017-09-19,jobs_archive_2017-09-20,jobs_archive_2017-09-21,jobs_archive_2017-09-22,jobs_archive_2017-09-23,jobs_archive_2017-09-24,jobs_archive_2017-09-25,jobs_archive_2017-09-26,jobs_archive_2017-09-27,jobs_archive_2017-09-28,jobs_archive_2017-09-29,jobs_archive_2017-09-30,jobs_archive_2017-10-01,jobs_archive_2017-10-02,jobs_archive_2017-10-03,jobs_archive_2017-10-04,jobs_archive_2017-10-05,jobs_archive_2017-10-06,jobs_archive_2017-10-07,jobs_archive_2017-10-08,jobs_archive_2017-10-09,jobs_archive_2017-10-10,jobs_archive_2017-10-11,jobs_archive_2017-10-12,jobs_archive_2017-10-13,jobs_archive_2017-10-14,jobs_archive_2017-10-15,jobs_archive_2017-10-16,jobs_archive_2017-10-17,jobs_archive_2017-10-18,jobs_archive_2017-10-19,jobs_archive_2017-10-20,jobs_archive_2017-10-21,jobs_archive_2017-10-22,jobs_archive_2017-10-23,jobs_archive_2017

In [3]:
my_query_ja={
    "size": 0,
    "_source": ["modificationhost", "cpuconsumptiontime","nevents","jeditaskid","actualcorecount","wall_time","nevents"],
  "query": {
    "bool": {
      "must": [
        {
          "term": {"jobstatus": "finished"},
          "term": {"computingsite":"BOINC_MCORE"}
        }
      ]
    }
  }
}
my_query_boinc={
    "size": 0,
    "_source": ["result_app_version_id","batch","cpu_time","host_name","elapsed_time"],
  "query": {
    "bool": {
      "must": [
        {
          "term": {"outcome":1},
          "term": {"result_app_id":14}
        }
      ]
    }
  }
}


def list_res(res):
    count=0
    for r in res:
        if count<=10:
            print(r)
            count+=1


#get_avg_boinc_power_by_task(tasks,res)
def get_ja_info(res):
    ja_info={}
    for r in res:
        if "actualcorecount" in r["_source"] and r["_source"]["actualcorecount"]:
            taskid=r["_source"]["jeditaskid"]
            cputime=r["_source"]["cpuconsumptiontime"]
            walltime=r["_source"]["wall_time"]*r["_source"]["actualcorecount"]
            nevents=r["_source"]["nevents"]
            ncore=r["_source"]["actualcorecount"]
            host=r["_source"]["modificationhost"]
            if type(host)!=str or host=="":
                continue
            host=re.sub(".*@","",host)
            if cputime>2000*nevents or cputime<=0:
                #print("bad cputime",cputime)
                continue
            if walltime>2000*nevents*ncore or walltime<=0:
                #print("bad walltime",walltime)
                continue
            cpu_eff=np.round(cputime/walltime,2)
            if cpu_eff<1:
                #print(host,cputime,walltime,ncore,cpu_eff)
                if not host in ja_info:
                    ja_info[host]={}
                if not taskid in ja_info[host]:
                    ja_info[host][taskid]={}
                    ja_info[host][taskid]["cputime"]=0
                    ja_info[host][taskid]["walltime"]=0
                    ja_info[host][taskid]["cpu_eff"]=0
                    ja_info[host][taskid]["cn"]=0
                    ja_info[host][taskid]["ncore"]=0
                if ja_info[host][taskid]["ncore"]!=0 and ja_info[host][taskid]["ncore"]!=ncore:
                    continue
                ja_info[host][taskid]["cputime"]+=cputime
                ja_info[host][taskid]["walltime"]+=walltime
                ja_info[host][taskid]["cn"]+=1
                ja_info[host][taskid]["cpu_eff"]+=cpu_eff
                ja_info[host][taskid]["ncore"]=ncore
    #print(ja_info)
    return ja_info
def get_boinc_info(res):
    boinc_info={}
    for r in res:
        taskid=r["_source"]["batch"]
        host=r["_source"]["host_name"]
        cputime=r["_source"]["cpu_time"]
        walltime=r["_source"]["elapsed_time"]
        av=r["_source"]["result_app_version_id"]
        if walltime<=0:
            continue
        cpu_eff=np.round(cputime/walltime,2)
       
        if host=="":
            continue
        if not host in boinc_info:
            boinc_info[host]={}
        if not taskid in boinc_info[host]:
            boinc_info[host][taskid]={}
            boinc_info[host][taskid]["cputime"]=0
            boinc_info[host][taskid]["walltime"]=0
            boinc_info[host][taskid]["cn"]=0
            boinc_info[host][taskid]["av"]=0
            boinc_info[host][taskid]["cpu_eff"]=0
        if boinc_info[host][taskid]["av"]!=0 and av!=boinc_info[host][taskid]["av"]:
            continue
         
        boinc_info[host][taskid]["cputime"]+=cputime
        boinc_info[host][taskid]["walltime"]+=walltime
        boinc_info[host][taskid]["cn"]+=1
        boinc_info[host][taskid]["av"]=av
        boinc_info[host][taskid]["cpu_eff"]+=cpu_eff
    #print(boinc_info)
    return boinc_info

res_ja = helpers.scan(es, query=my_query_ja, index=ind_ja, scroll='5m', timeout="5m", size=1000)
ja_info=get_ja_info(res_ja)

res_boinc = helpers.scan(es, query=my_query_boinc, index=ind_boinc, scroll='5m', timeout="5m", size=1000)
 
boinc_info=get_boinc_info(res_boinc)    



In [4]:


dlist=[]
head=["av","ncore","taskid","host","panda_cn","boinc_cn","panda_cpusec_perjob","panda_cpu_eff","boinc_cpu_eff",\
      "cputime_diff","walltime_diff","job_cpu_eff"]
for host in ja_info.keys():
    for taskid in ja_info[host].keys():
        if host in boinc_info and taskid in boinc_info[host]:
            av=boinc_info[host][taskid]["av"]
            ncore=ja_info[host][taskid]["ncore"]
            boinc_cn=boinc_info[host][taskid]["cn"]
            ja_cn=ja_info[host][taskid]["cn"]
            if boinc_cn!=ja_cn:
                continue
            boinc_walltime=ncore*boinc_info[host][taskid]["walltime"]/boinc_cn
            panda_walltime=ja_info[host][taskid]["walltime"]/ja_cn
            boinc_cputime=boinc_info[host][taskid]["cputime"]/boinc_cn
            panda_cputime=ja_info[host][taskid]["cputime"]/ja_cn
            cpu_eff=ja_info[host][taskid]["cpu_eff"]/ja_cn
            cputime_perjob=ja_info[host][taskid]["cputime"]/ja_cn
            boinc_cpu_eff=boinc_info[host][taskid]["cpu_eff"]/boinc_cn
            boinc_cpu_eff=np.round(boinc_cpu_eff/ncore,2)
            job_cpu_eff=np.round(panda_cputime/boinc_walltime,2)
            if boinc_walltime<=0:
                continue
            if boinc_cpu_eff>cpu_eff:
                continue
            if boinc_walltime<panda_walltime-600:
                print("bad boinc walltime",boinc_walltime-panda_walltime)
                #boinc_walltime=panda_walltime
                continue
            if boinc_cputime<panda_cputime-600:
                print("bad boinc cputime",boinc_cputime-panda_cputime)
                #boinc_cputime=panda_cputime
                continue
            t=[av,ncore,taskid,host,boinc_cn,ja_cn,cputime_perjob,cpu_eff,boinc_cpu_eff,\
               boinc_cputime-panda_cputime,boinc_walltime-panda_walltime,job_cpu_eff]    
            dlist.append(t)
df=pd.DataFrame(dlist,columns=head)


df.to_csv("boinc_perf.csv")
df=df.sort_values(by="av",ascending=False)
print(df.describe())



bad boinc walltime -8503.617052800008
bad boinc walltime -12299.88046
bad boinc walltime -264046.527846
bad boinc cputime -1249.4400000000023
bad boinc walltime -1612.0725325000021
bad boinc cputime -3331.3100000000013
bad boinc cputime -4445.356666666667
bad boinc cputime -32231.520000000004
bad boinc cputime -751.2000000000116
bad boinc cputime -5181.955000000002
bad boinc cputime -26107.68333333332
bad boinc cputime -16994.0
bad boinc cputime -53568.28
bad boinc cputime -1532.6650000000009
bad boinc cputime -1148.9599999999991
bad boinc walltime -28282.866836
bad boinc walltime -42440.074279500004
bad boinc cputime -19785.434666666668
bad boinc walltime -1533.1644114285737
bad boinc cputime -70663.5
bad boinc walltime -2173.6395239999983
bad boinc walltime -166213.685456
bad boinc cputime -2792.584999999999
bad boinc cputime -23554.706000000006
bad boinc cputime -18408.435000000005
bad boinc walltime -643.735268000004
bad boinc cputime -919.1999999999971
bad boinc cputime -2079.3928